<a href="https://colab.research.google.com/github/Lakshmi-Ganesh-Mulluri/MAVERICKS/blob/main/ML_Maverick's_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy scikit-learn tensorflow gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
def preprocess_for_lstm(df, time_steps=60):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)
    X, y = [], []
    for i in range(time_steps, len(scaled_data)):
        X.append(scaled_data[i-time_steps:i])
        y.append(scaled_data[i])
    return np.array(X), np.array(y), scaler

In [4]:
def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [5]:
def assess_risk(age, experience):
    if age < 30 and experience == "high":
        return "aggressive"
    elif age > 50 or experience == "low":
        return "conservative"
    return "moderate"

In [6]:
def recommend_portfolio(risk):
    return {
        "aggressive": {"Tech": 50, "Emerging Markets": 30, "Bonds": 20},
        "moderate": {"Tech": 30, "Bonds": 40, "Blue-chip": 30},
        "conservative": {"Bonds": 60, "Dividends": 30, "Gold": 10}
    }[risk]

In [7]:
def train_and_predict(csv_file):
    try:
        df = pd.read_csv(csv_file.name if hasattr(csv_file, '/content/Bitcoin  Stock Market Analysis.csv') else csv_file)
        if 'Close' not in df.columns:
            return "❌ CSV must contain a 'Close' column."

        df = df[['Close']].dropna()
        if len(df) < 70:
            return "❌ Please upload a CSV with at least 70 rows of 'Close' prices."

        X, y, scaler = preprocess_for_lstm(df)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        model = create_lstm_model((X.shape[1], 1))
        model.fit(X, y, epochs=5, batch_size=32, verbose=0)

        X_latest = X[-1].reshape((1, X.shape[1], 1))
        pred_scaled = model.predict(X_latest)[0][0]
        pred_price = scaler.inverse_transform([[pred_scaled]])[0][0]

        return f"✅ Predicted next close price: ${pred_price:.2f}"

    except Exception as e:
        return f"❌ Error: {str(e)}"

In [8]:
def get_portfolio(age, experience):
    risk = assess_risk(age, experience)
    portfolio = recommend_portfolio(risk)
    result = f"🎯 Risk Profile: {risk.capitalize()}\n📊 Suggested Portfolio:\n"
    for asset, percent in portfolio.items():
        result += f"- {asset}: {percent}%\n"
    return result

In [9]:
with gr.Blocks() as demo:
    gr.Markdown("#           📈 AI Stock Market Assistant         ")

    with gr.Tab("📊 Predict Stock Price"):
        file_input = gr.File(label="Upload a File.csv", file_types=[".csv"])
        predict_button = gr.Button("Train & Predict")
        output_text = gr.Textbox(label="Prediction Result")
        predict_button.click(fn=train_and_predict, inputs=file_input, outputs=output_text)

    with gr.Tab("💼 Portfolio Recommendation"):
        age_input = gr.Number(label="Enter your age", value=35)
        experience_input = gr.Dropdown(["low", "medium", "high"], label="Investment Experience", value="medium")
        recommend_button = gr.Button("Get Portfolio Suggestion")
        recommend_output = gr.Textbox(label="Recommendation")
        recommend_button.click(fn=get_portfolio, inputs=[age_input, experience_input], outputs=recommend_output)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d5b4caf1489cc03ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
